In [1]:
from __future__ import division, print_function
%matplotlib inline
from importlib import reload  # Python 3
import utils; reload(utils)
from utils import *
from keras.layers.merge import dot, add, concatenate

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using TensorFlow backend.


In [2]:
#path = "data/ml-latest-small/"  # from https://grouplens.org/datasets/movielens/
#path = "data/ml-20m/"
path = "datas/movielens/" #from https://github.com/ChicagoBoothML/DATA___MovieLens___20M
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

batch_size=64
#batch_size=1

In [3]:
#20M dataset
#ratings = pd.read_csv(path+'ratings.csv')

#2M dataset
#ratings = pd.read_csv(path+'ratings01.csv')

#100k dataset
ratings = pd.read_csv(path+'ratings100k.csv')

In [4]:
#show 5 first file rows
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [5]:
#nb of lines
len(ratings)

100000

## datas prep
1. read files : ratings + movies
1. get list of unique items in ratings : users ID & movies ID
1. reorder users ID & movies ID to have countinuous integer items

In [6]:
#read film names
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict

#https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_dict.html
#https://stackoverflow.com/questions/30060616/python-pandas-to-dict-function

In [7]:
movie_names1 = pd.read_csv(path+'movies.csv').set_index('movieId')['title']
print(movie_names1[:5])

movieId
1                      Toy Story (1995)
2                        Jumanji (1995)
3               Grumpier Old Men (1995)
4              Waiting to Exhale (1995)
5    Father of the Bride Part II (1995)
Name: title, dtype: object


In [8]:
movie_names2 = pd.read_csv(path+'movies.csv').set_index('movieId')
print(movie_names2[:5])

                                      title  \
movieId                                       
1                          Toy Story (1995)   
2                            Jumanji (1995)   
3                   Grumpier Old Men (1995)   
4                  Waiting to Exhale (1995)   
5        Father of the Bride Part II (1995)   

                                              genres  
movieId                                               
1        Adventure|Animation|Children|Comedy|Fantasy  
2                         Adventure|Children|Fantasy  
3                                     Comedy|Romance  
4                               Comedy|Drama|Romance  
5                                             Comedy  


In [9]:
usersUniq = ratings.userId.unique()
moviesUniq = ratings.movieId.unique()

print(usersUniq[20:])
print(moviesUniq[20:])

[ 21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95
  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120
 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195
 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220
 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245
 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 26

In [10]:
# userId and movieId become ditionary elements with values ranging from 0 to max len 
userUniqID2idx = {userID:index for index,userID in enumerate(usersUniq)}
movieUniqID2idx = {movieID:index for index,movieID in enumerate(moviesUniq)}

In [12]:
ratings.movieId[:20]

0       2
1      29
2      32
3      47
4      50
5     112
6     151
7     223
8     253
9     260
10    293
11    296
12    318
13    337
14    367
15    541
16    589
17    593
18    653
19    919
Name: movieId, dtype: int64

In [22]:
#We update the movie and user ids so that they are contiguous integers, which we want when using embeddings.
ratings.movieId = ratings.movieId.apply(lambda x: movieUniqID2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userUniqID2idx[x])

#https://chrisalbon.com/python/data_wrangling/pandas_apply_operations_to_dataframes/

In [23]:
ratings.movieId[:15]

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
Name: movieId, dtype: int64

In [24]:
ratings.movieId[200:220]

200    192
201    193
202    194
203    195
204    196
205    197
206    198
207    199
208    200
209    201
210     80
211    202
212    203
213    204
214    205
215    206
216    207
217    208
218    209
219    210
Name: movieId, dtype: int64

In [25]:
ratings.loc[(ratings['movieId'] == 80)] 

,userId,movieId,rating,timestamp
80,0,80,4.0,1094785777
210,1,80,2.0,974820748
1101,10,80,5.0,1230853841
2521,22,80,3.0,914457821
4648,44,80,4.0,1161818031
5025,49,80,5.0,1182348358
5812,53,80,3.0,974836859
6237,55,80,3.0,967240554
6690,57,80,4.5,1144058968
7149,60,80,3.5,1197164241


In [26]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())

In [27]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()

In [28]:
user_min, user_max, movie_min, movie_max, n_users, n_movies

(0, 701, 0, 8226, 702, 8227)

### embeddings

In [28]:
#number of latent factors in each embedding
n_factors = 50

In [29]:
np.random.seed = 42

#https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.seed.html

In [30]:
#Randomly split into training and validation

#Create an array of the given shape and populate it with random samples from a uniform distribution over [0, 1).
msk = np.random.rand(len(ratings)) < 0.8
print(msk)

trn = ratings[msk]
val = ratings[~msk]

[False  True  True ...,  True  True  True]


In [31]:
trn[:5]

,userId,movieId,rating,timestamp
1,0,1,3.5,1112484676
2,0,2,3.5,1112484819
3,0,3,3.5,1112484727
4,0,4,3.5,1112484580
5,0,5,3.5,1094785740


In [32]:
val[:5]

,userId,movieId,rating,timestamp
0,0,0,3.5,1112486027
6,0,6,4.0,1094785734
13,0,13,3.5,1094785709
14,0,14,3.5,1112485980
16,0,16,3.5,1112485557


### Create subset for Excel

In [33]:
#We create a crosstab of the most popular movies and most movie-addicted users 
#which we'll copy into Excel for creating a simple example. 
#This isn't necessary for any of the modeling below however.
g=ratings.groupby('userId')['rating'].count()
topUsers=g.sort_values(ascending=False)[:15]

In [34]:
topUsers

userId
155    2179
585    1431
571    1326
358    1300
207    1288
393    1212
297    1127
115    1110
631    1094
613    1042
103     998
423     918
647     904
586     873
347     786
Name: rating, dtype: int64

In [35]:
top_r = ratings.join(topUsers, rsuffix='_r', how='inner', on='userId')

In [36]:
top_r.head(15)

,userId,movieId,rating,timestamp,rating_r
11454,103,1187,1.0,944917003,998
11455,103,387,3.0,944928813,998
11456,103,630,3.0,945002688,998
11457,103,681,4.0,945003357,998
11458,103,2021,3.0,944917299,998
11459,103,1,3.0,944920275,998
11460,103,3163,3.0,944924853,998
11461,103,2,3.0,945002402,998
11462,103,1045,4.0,944928872,998
11463,103,1046,3.0,944912818,998


In [37]:
g=ratings.groupby('movieId')['rating'].count()
topMovies=g.sort_values(ascending=False)[:15]

In [38]:
topMovies

movieId
11     350
369    340
12     305
182    302
17     295
9      264
178    262
16     256
331    253
652    247
227    241
205    238
392    235
242    234
237    233
Name: rating, dtype: int64

In [39]:
top_r = top_r.join(topMovies, rsuffix='_r', how='inner', on='movieId')

In [40]:
top_r.head(15)

,userId,movieId,rating,timestamp,rating_r,rating_r
11488,103,392,2.0,945002463,998,235
13216,115,392,3.0,1132711598,1110,235
19908,155,392,5.0,1039204398,2179,235
25897,207,392,5.0,942448811,1288,235
38535,297,392,3.0,992545882,1127,235
46961,358,392,4.0,1087076743,1300,235
53882,393,392,4.0,957483687,1212,235
78348,571,392,5.0,1324264879,1326,235
81197,585,392,3.0,1112498213,1431,235
86881,613,392,3.5,1122836727,1042,235


In [41]:
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum)

movieId,9,11,12,16,17,178,182,205,227,237,242,331,369,392,652
userId,,,,,,,,,,,,,,,
103,3.0,3.0,3.0,NaN,3.0,NaN,NaN,2.0,NaN,2.0,NaN,1.0,NaN,2.0,5.0
115,4.5,4.5,4.5,4.0,3.0,4.5,4.0,4.5,3.0,3.0,1.0,4.0,4.0,3.0,4.0
155,NaN,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
207,4.5,5.0,4.5,2.5,4.5,3.5,4.5,3.5,4.0,5.0,4.0,3.0,1.0,5.0,4.5
297,5.0,4.0,5.0,5.0,4.0,5.0,4.0,5.0,4.0,4.0,4.0,5.0,3.0,3.0,3.0
347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.5,NaN,NaN,NaN,4.5
358,5.0,5.0,5.0,5.0,5.0,3.5,4.0,4.0,5.0,4.0,3.0,3.0,4.5,4.0,5.0
393,4.0,5.0,3.0,4.0,5.0,2.5,2.0,5.0,NaN,4.0,1.0,5.0,NaN,4.0,3.0
423,3.5,NaN,4.5,3.5,NaN,4.5,4.0,NaN,NaN,NaN,4.0,4.5,3.0,NaN,NaN
